In [ ]:
import os
os.environ["SSL_CERT_FILE"] = "Fortinet_CA_SSL(15).cer"


In [ ]:
from mistralai import Mistral
from mistralai.models import UserMessage
from dotenv import load_dotenv
import os
import json

# Preparation

### Generate fake weather data

In [ ]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

In [ ]:
get_weather('Cairo')

# Q1. Define function description

In [ ]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get the temperature for a given city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Name the city to get its temperature."
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

In [ ]:
get_weather_tool = {
    "type": "function",
    "function": {           
        "name": "get_weather",
        "description": "Get the temperature for a given city",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "Name the city to get its temperature."
                }
            },
            "required": ["city"],
            "additionalProperties": False
        }
    }
}


In [ ]:
load_dotenv()

In [ ]:
api_key = os.getenv("API_KEY")

In [ ]:
client = Mistral(api_key = api_key )

In [ ]:
question = "What's the current temperature in Berlin, Germany?"

system_prompt = """
You are a weather program. 
Your task is to fetch the current temperature for the specified city in Celsius.
The city will be provided by the user. Make sure to only return the temperature.
""".strip()

tools = [get_weather_tool]

chat_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question}
]

response = client.chat.complete(
    model = "mistral-medium-latest",
    messages = chat_messages, 
    tools = tools,
    tool_choice = "any", 
    parallel_tool_calls = False,
)
response.choices



In [ ]:
calls = response.choices
call = calls[0]
f_name = call.message.tool_calls[0].function.name
arguments = json.loads(call.message.tool_calls[0].function.arguments)

In [ ]:
f = globals()[f_name]

In [ ]:
search_results = f(**arguments)
search_results

# Q2. Adding another tool

In [ ]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [ ]:
# Add description 

set_weather_tool = {
    "type": "function",
    "function": {
        "name": "set_weather",
        "description": "Add or update the temperature (°C) for a specified city in the weather database.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "Name of the city to add or update in the weather database"
                },
                "temp": {
                    "type": "number",
                    "description": "Temperature in Celsius to set for the city"
                }
            },
        "required": ["city", "temp"],
        "additionalProperties": False
        }
    }
}



In [ ]:
question = "What's the current temperature in berlin?"

system_prompt = """
You are a data assistant. 
Your task is to fetch the current temperature from get_weather_tool.
save the result in the database.
""".strip()

tools = [get_weather_tool, set_weather_tool]

chat_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question}
]

response = client.chat.complete(
    model = "mistral-medium-latest",
    messages = chat_messages, 
    tools = tools,
    tool_choice = "any", 
    parallel_tool_calls = False,
)
response.choices



# Q3. Install FastMCP

In [ ]:
pip install fastmcp --trusted-host pypi.org --trusted-host files.pythonhosted.org

In [ ]:
!fastmcp --version 

# Q4. Simple MCP Server

The code have been saved in weather_server.py and run it by (fastmcp run weather_server.py)

and the output is 

╭─ FastMCP 2.0 ──────────────────────────────────────────────────────────────╮
│                                                                            │
│        _ __ ___ ______           __  __  _____________    ____    ____     │
│       _ __ ___ / ____/___ ______/ /_/  |/  / ____/ __ \  |___ \  / __ \    │
│      _ __ ___ / /_  / __ `/ ___/ __/ /|_/ / /   / /_/ /  ___/ / / / / /    │
│     _ __ ___ / __/ / /_/ (__  ) /_/ /  / / /___/ ____/  /  __/_/ /_/ /     │
│    _ __ ___ /_/    \__,_/____/\__/_/  /_/\____/_/      /_____(_)____/      │
│                                                                            │
│                                                                            │
│                                                                            │
│    🖥️   Server name:     Demo 🚀                                             │
│    📦 Transport:       STDIO                                               │
│                                                                            │
│    📚 Docs:            https://gofastmcp.com                               │
│    🚀 Deploy:          https://fastmcp.cloud                               │
│                                                                            │
│    🏎️   FastMCP version: 2.10.5                                              │
│    🤝 MCP version:     1.11.0                                              │
│                                                                            │
╰────────────────────────────────────────────────────────────────────────────╯


[07/15/25 13:20:11] INFO     Starting MCP server 'Demo 🚀' with transport 'stdio

# Q5. Protocol

{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": {"city": "Berlin"}}}


OUTPUT:


{"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"20.0"}],"structuredContent":{"result":20.0},"isError":false}}

# Q6. Client

In [3]:
import asyncio
from fastmcp import Client
import weather_server

In [8]:
async def main():
    async with Client("weather_server") as mcp_client:
        tools = await mcp_client.get_tools()
        print("Available tools:")
        for tool in tools:
            print(tool)

await main()  

ValueError: Could not infer a valid transport from: weather_server